# Compare RMG to the Grabow model

In [ ]:
import os
import glob
import itertools
from random import randint
import pandas as pd
import numpy as np
import csv
import cantera as ct
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn
plt.style.use('seaborn-white')
from pathlib import Path
import time

import matplotlib.cm as cm
from IPython.display import Image
import git

In [ ]:
from rmgpy import chemkin

## Git-python: checkout master branch

In [ ]:
# RMG model path that is being analyzed. cantera analysis have a folder name 
# based off of the commit message, hash, and date
rmg_model_path = "/work/westgroup/ChrisB/_01_MeOH_repos/RMG_run_comparisons/bep_parameter_study/rmg_runs/meoh_main/"
# rmg_model_path = "/work/westgroup/ChrisB/_01_MeOH_repos/meOH-synthesis/"
repo = git.Repo(rmg_model_path)
date = time.localtime(repo.head.commit.committed_date)
git_date = time.strftime("%Y_%m_%d_%H%M", date)
git_sha = str(repo.head.commit)[0:6]
git_msg = str(repo.head.commit.message)[0:50].replace(" ", "_").replace("'", "_").replace("\n", "")
git_file_string = f"{git_date}_{git_sha}_{git_msg}"
commit = git_file_string
commit

## Load a list of reactions for each species generation rate

In [ ]:
cti_file = rmg_model_path + "base/cantera/chem_annotated.cti"

gas = ct.Solution(cti_file)
surface = ct.Interface(cti_file, "surface1", [gas])

model_dict_file = rmg_model_path + "base/chemkin/species_dictionary.txt"
grabow_dict_file = "./species_data/species_dictionary.txt"

In [ ]:
model_dict = chemkin.load_species_dictionary(model_dict_file)
grabow_dict = chemkin.load_species_dictionary(grabow_dict_file)

In [ ]:
model_dict.keys()

In [ ]:
# make a dictionary to "translate" the names from the grabow model to ours
# irrespective of the naming convention.
spc_trans = {}
for name, entry in model_dict.items(): 
    for g_name, g_entry in grabow_dict.items():
        if entry.is_isomorphic(g_entry):
            # remove (#) so it is neater
            g_name_new = g_name.split("(", 1)[0]
            spc_trans.update({g_name_new :name})
spc_trans                   

In [ ]:
# read last line from each CSV file
data_dict = {}

# exclude CSP, steady state, and non - sensitivity data
exclude = ["/csp/", "/transient/", "Graaf_experiment_comparison", "analysis_scripts"]

# ensure we don't grab any files that have just printed the 
#column names
min_file_size = 5000

first_file = True
for path in Path(f'./{commit}').rglob('*.csv'):
    path_str = str(path)
    if all(f not in path_str for f in exclude):
        if os.stat(path_str).st_size >= min_file_size:
            if first_file == True: 
                data = pd.read_csv(path_str)
                aggdata = data.tail(1)
                first_file = False
            else: 
                data = pd.read_csv(path_str)
                aggdata = aggdata.append(data.tail(1))

                
print(f"{len(aggdata)} successful runs in commit:\n{commit}")
data_dict.update({commit:aggdata})

## get reactions for each one listed in TOF chart in grabow paper

     Grabow study name      |  Cantera label or formula
     --------------------------------------------------
     "Methanol Production"  |  methanol net production rate  
     "Water-Gas Shift"      |  OH* + CO* -> COOH* + * : leave as-is
     "CO Hydrogenation"     |  Any reaction where CO receives an H and goes to HCO*. 
     "CO2 Hydrogenation"    |  Any reaction where CO2 receives an H and goes to HCO2*
     "H2O Production"       |  rate of H2O*

In [ ]:
co_co2_label = "CO2/(CO2+CO)"

In [ ]:
for index,rxn in enumerate(surface.reactions()):
    if spc_trans["H2O"] in rxn.equation:
        surface.set_multiplier(0,index)


functions for grabbing ROP data

### methanol

In [ ]:
rop_str = " ROP [kmol/m^2 s]"        
ch3oh_dict = {}  
        
for rxn in surface.reactions():
    if spc_trans["CH3OH"] in rxn.equation: 
        rxn_string = rxn.equation + rop_str
        if spc_trans["CH3OH"] in rxn.product_string:
            print(rxn.equation, "forward")
            ch3oh_dict.update({rxn_string:1})
        elif spc_trans["CH3OH"] in rxn.reactant_string: 
            print(rxn.equation, "reverse")
            ch3oh_dict.update({rxn_string:-1})

In [ ]:
aggdata["Methanol TOF ($s^{-1}$)"] = 0 #(initialize)
for rxn, direct in ch3oh_dict.items():
    if direct == 1:
        aggdata["Methanol TOF ($s^{-1}$)"] += aggdata[rxn]/surface.site_density
    elif direct == -1:
        aggdata["Methanol TOF ($s^{-1}$)"] -= aggdata[rxn]/surface.site_density

### Water-Gas Shift
overall: $CO + H_2O <--> CO_2 + H_2$ 



In [ ]:
wgs_hyd_dict = {}
for rxn in surface.reactions():
    if spc_trans["CO*"] in rxn.reactant_string and spc_trans["OH*"] in rxn.reactant_string: 
        rxn_string = rxn.equation + rop_str
        print(rxn.equation, "forward")
#         wgs_hyd_dict.update({rxn_string:1})

In [ ]:
# hardcode HOX(19) + OCX(17) <=> HOCXO(23) + X(1) for now just to analyze
wgs_hyd_dict.update({"HOX(19) + OCX(17) <=> HOCXO(23) + X(1) ROP [kmol/m^2 s]":1})

In [ ]:
aggdata["WGS Reaction TOF ($s^{-1}$)"] = 0 #(initialize)
for rxn, direct in wgs_hyd_dict.items():
    if direct == 1:
        aggdata["WGS Reaction TOF ($s^{-1}$)"] += aggdata[rxn]/surface.site_density
    elif direct == -1:
        aggdata["WGS Reaction TOF ($s^{-1}$)"] -= aggdata[rxn]/surface.site_density

 ### CO hydrogenation

In [ ]:
co_hyd_dict = {}
rop_str = " ROP [kmol/m^2 s]"
for rxn in surface.reactions():
    if spc_trans["CO*"] in rxn.equation and spc_trans["HCO*"] in rxn.equation: 
        rxn_string = rxn.equation + rop_str
        if spc_trans["CO*"] in rxn.reactant_string and not spc_trans["HCO*"] in rxn.reactant_string:
            print(rxn.equation, "forward")
            co_hyd_dict.update({rxn_string:1})
        elif spc_trans["CO*"] in rxn.product_string and not spc_trans["HCO*"] in rxn.product_string: 
            print(rxn.equation, "reverse")
            co_hyd_dict.update({rxn_string:-1})

In [ ]:
aggdata["CO Hydrogenation TOF ($s^{-1}$)"] = 0 #(initialize)
for rxn, direct in co_hyd_dict.items():
    if direct == 1:
        aggdata["CO Hydrogenation TOF ($s^{-1}$)"] += aggdata[rxn]/surface.site_density
    elif direct == -1:
        aggdata["CO Hydrogenation TOF ($s^{-1}$)"] -= aggdata[rxn]/surface.site_density

### CO2 hydrogenation

In [ ]:
co2_hyd_dict = {}
rop_str = " ROP [kmol/m^2 s]"
for rxn in surface.reactions():
    if spc_trans["CO2*"] in rxn.equation and spc_trans["HCOO*"] in rxn.equation: 
        rxn_string = rxn.equation + rop_str
        if spc_trans["CO2*"] in rxn.reactant_string and not spc_trans["HCOO*"] in rxn.reactant_string:
            print(rxn.equation, "forward")
            co2_hyd_dict.update({rxn_string:1})
        elif spc_trans["CO2*"] in rxn.product_string and not spc_trans["HCOO*"] in rxn.product_string: 
            print(rxn.equation, "reverse")
            co2_hyd_dict.update({rxn_string:-1})

In [ ]:
aggdata["CO2 Hydrogenation TOF ($s^{-1}$)"] = 0 #(initialize)
for rxn, direct in co2_hyd_dict.items():
    if direct == 1:
        aggdata["CO2 Hydrogenation TOF ($s^{-1}$)"] += aggdata[rxn]/surface.site_density
    elif direct == -1:
        aggdata["CO2 Hydrogenation TOF ($s^{-1}$)"] -= aggdata[rxn]/surface.site_density

### H2O production

In [ ]:
h2o_dict = {}        
for rxn in surface.reactions():
    if spc_trans["H2O"] in rxn.equation: 
        rxn_string = rxn.equation + rop_str
        if spc_trans["H2O"] in rxn.reactant_string:
            print(rxn.equation, "reverse")
            h2o_dict.update({rxn_string:-1})
        elif spc_trans["H2O"] in rxn.product_string: 
            print(rxn.equation, "forward")
            h2o_dict.update({rxn_string:1}) 

In [ ]:
aggdata["H2O TOF ($s^{-1}$)"] = 0 #(initialize)
for rxn, direct in h2o_dict.items():
    if direct == 1:
        aggdata["H2O TOF ($s^{-1}$)"] += aggdata[rxn]/surface.site_density
    elif direct == -1:
        aggdata["H2O TOF ($s^{-1}$)"] -= aggdata[rxn]/surface.site_density

### surface site density ($\frac{kmol}{m^2}$)

In [ ]:
surface.site_density
site_dens_mol = surface.site_density*1e-3
site_dens_mol

### TOF Calculation from ROP: 

$ROP\left[\frac{kmoles}{m^{2}*s}\right] * \frac{1}{\Gamma} \left[\frac{m^2}{kmol}\right] = TOF \left[\frac{1}{sec}\right]$   

## Turn over frequency comparison

Compare turn over frequencies for reaction(s) in the methanol model



In [ ]:
Image('../images/Grabow_plots/Grabow_TOF.png',width = 700, height = 300)

### Load Grabow Data

In [ ]:
Grabow_rates = pd.read_csv("../Grabow_data/Paper_plot_data/Grabow_rates.csv")
Grabow_cov = pd.read_csv("../Grabow_data/Paper_plot_data/Grabow_coverages.csv")

### Make TOF plot functions

In [ ]:
def plot_rates_grabow(df):
    '''
    This function returns a 2x2 set of charts like the ones above, 
    for comparison with the grabow rates and coverages. 
    df - dataframe to load
    labels - list of labels to use. first is the "x" values, then the y
    '''
    h2_mol = [0.5, 0.75, 0.8, 0.95]
    grabow_labels = [
        "Methanol Production",
        "Water-Gas Shift",
        "CO Hydrogenation",
        "CO2 Hydrogenation",
        "H2O Production",]
    
    h2_label = "y(H2)"
    co_co2_label = "CO2/(CO+CO2)"
    fig, ax = plt.subplots(2,2,figsize=(15,15))
    
    axes = [(0,0), (0,1), (1,0), (1,1)]
    
    color_dict = { 0:"k", 1:"y", 2:"g", 3:"b", 4:"r"}

    for coord,h2 in enumerate(h2_mol):
        
        for color,label in enumerate(grabow_labels):
            # "slot" where we place chart
            slot = axes[coord]
            df[np.isclose(df["y(H2)"], h2)].plot(x=co_co2_label,
                                                 y=label,
                                                 ax=ax[slot], 
                                                 color=color_dict[color],
                                                )
            ax[slot].set_title(f'mole frac H2 = {h2}')
            ax[slot].autoscale(enable=True, axis='y')
            ax[slot].set_ylabel("Turn over frequency ($s^{-1}$)")

def plot_rates_rmg(df):
    '''
    This function returns a 2x2 set of charts like the ones above, 
    for comparison with the grabow rates and coverages. 
    df - dataframe to load
    labels - list of labels to use. first is the "x" values, then the y
    '''
    
    h2_mol = [0.5, 0.75, 0.8, 0.95]
    # this part is tricky. 
    
    #         "Methanol Production"  :  CH3O* + H* -> CH3OH* + *;
    #         "Water-Gas Shift"      :  OH* + CO* -> COOH* + *
    #         "CO Hydrogenation"     :  CO* + H* -> HCO* + *
    #         "CO2 Hydrogenation"    :  CO2* + H* -> HCO2* + *
    #         "H2O Production"       :  #1 - #2
                                        #1:  H2O*+*->OH*+H*
    #                                   #2:  COOH* + OH* -> CO2* + H2O*
    rmg_labels = [
        "Methanol TOF ($s^{-1}$)",
        "WGS Reaction TOF ($s^{-1}$)",
        "CO Hydrogenation TOF ($s^{-1}$)",
        "CO2 Hydrogenation TOF ($s^{-1}$)",
        "H2O TOF ($s^{-1}$)",
    ]
        
    h2_label = "X_h2 initial"
    co_co2_label = "CO2/(CO2+CO)"
    color_dict = { 0:"k", 1:"y", 2:"g", 3:"b", 4:"r"}
    fig, ax = plt.subplots(2,2,figsize=(15,15))
    axes = [(0,0), (0,1), (1,0), (1,1)]

    for coord,h2 in enumerate(h2_mol):
        
        for color,label in enumerate(rmg_labels):
            # "slot" where we place chart
            slot = axes[coord]
            df[np.isclose(df[h2_label], h2)].plot(x=co_co2_label,
                                                 y=label,
                                                 ax =ax[slot], 
                                                 color=color_dict[color],
                                                )
            ax[slot].set_title(f'mole frac H2 = {h2}')
            ax[slot].autoscale(enable=True, axis='y')
            ax[slot].set_ylabel("Turn over frequency ($s^{-1}$)")



### Plot TOFs

In [ ]:
plot_rates_grabow(Grabow_rates)

In [ ]:
plot_rates_rmg(aggdata[aggdata["T (K)"] == 528])

## Coverage comparison

Look at the surface coverage and gas phase concentrations for key species

## Plot coverage functions

In [ ]:
def plot_covs_grabow(df):
    '''
    This function returns a 2x2 set of charts like the ones above, 
    for comparison with the rmg rates and coverages. 
    df - dataframe to load
    '''
    
    h2_mol = [0.5, 0.75, 0.8, 0.95]
    grabow_labels = [
        "vacant",
        "HCOO",
        "CH3O",
        "H",
        "OH",
    ]
    
    h2_label = "y(H2)"
    co_co2_label = "CO2/(CO+CO2)"
    fig, ax = plt.subplots(2,2,figsize=(15,15))
    
    axes = [(0,0), (0,1), (1,0), (1,1)]
    
    color_dict = { 0:"k", 1:"y", 2:"g", 3:"b", 4:"r"}

    for coord,h2 in enumerate(h2_mol):
        
        for color,label in enumerate(grabow_labels):
            # "slot" where we place chart
            slot = axes[coord]
            df[np.isclose(df["y(H2)"], h2)].plot(x=co_co2_label,
                                                 y=label,
                                                 ax=ax[slot], 
                                                 color=color_dict[color],
                                                )
            ax[slot].set_title(f'mole frac H2 = {h2}')
            ax[slot].autoscale(enable=True, axis='y')
            ax[slot].set_ylabel("site fraction")

def plot_covs_rmg(df, labels, gas=False):
    '''
    This function returns a 2x2 set of charts like the ones above, 
    for comparison with the grabow rates and coverages. 
    df - dataframe to load
    labels - list of labels to use. first is the "x" values, then the y
    '''
    
    h2_mol = [0.5, 0.75, 0.8, 0.95]
    
    rmg_labels = labels
        
    h2_label = "X_h2 initial"
    co_co2_label = "CO2/(CO2+CO)"
    color_dict = { 0:"k", 1:"y", 2:"g", 3:"b", 4:"r", 5:"deeppink"}
    fig, ax = plt.subplots(2,2,figsize=(15,15))
    axes = [(0,0), (0,1), (1,0), (1,1)]

    for coord,h2 in enumerate(h2_mol):
        
        for color,label in enumerate(rmg_labels):
            # "slot" where we place chart
            slot = axes[coord]
            df[np.isclose(df[h2_label], h2)].plot(x=co_co2_label,
                                                 y=label,
                                                 ax =ax[slot], 
                                                 color=color_dict[color],
                                                )
            ax[slot].set_title(f'mole frac H2 = {h2}')
            ax[slot].autoscale(enable=True, axis='y')
            if gas: 
                ax[slot].set_ylabel("mole fraction")
            else: 
                ax[slot].set_ylabel("site fraction")

In [ ]:
plot_covs_grabow(Grabow_cov)

In [ ]:
labels = [
    spc_trans["CO"],
    spc_trans["CO2"],
    spc_trans["H2O"],
    spc_trans["CH3OH"],
    spc_trans["H2"],
]

plot_covs_rmg(aggdata[aggdata["T (K)"] == 528], labels, gas=True)

In [ ]:
labels = [
    spc_trans["X"],
    spc_trans["HCOO*"],
    spc_trans["CH3O*"],
    spc_trans["H*"],
    spc_trans["OH*"],
    spc_trans["H2O*"]
]

plot_covs_rmg(aggdata[aggdata["T (K)"] == 528], labels)

In [ ]:
# get the maximum values
max_dist = 2
imp_species = []

h2 = 0.5
co_co2 = 0.5


df_short = aggdata[(aggdata["T (K)"] == 528) & \
                   np.isclose(aggdata["X_h2 initial"],h2) & \
                   np.isclose(aggdata["CO2/(CO2+CO)"],co_co2)]
thresh = 1e-6

print(f'Species Steady state concentrations that go above :{thresh}', '\n')

for i in range (15, len(df_short.columns)):

    column = df_short[df_short.columns[i]]
    max_value = column.max()
    max_index = column.idxmax()

    if max_value >= thresh and (("X" in df_short.columns[i]) or \
                              ("Pt" in df_short.columns[i]) or \
                              ("Pd" in df_short.columns[i]) or \
                              ("*" in df_short.columns[i])) \
    and not "ROP" in df_short.columns[i] \
    and not "sensitivity" in df_short.columns[i]:

        imp_species.append(df_short.columns[i])
        print(df_short.columns[i], "Max = " "{:.10f}".format(max_value))

In [ ]:
# get the maximum values
max_dist = 2
imp_rxns = {}

h2 = 0.5
co_co2 = 0.5

df_short = aggdata[(aggdata["T (K)"] == 528) & \
                   np.isclose(aggdata["X_h2 initial"],h2) & \
                   np.isclose(aggdata["CO2/(CO2+CO)"],co_co2)]

df_short
thresh = 1e-15

print(f'Species Steady state ROPs that go above :{thresh}', '\n')

for i in range (15, len(df_short.columns)):

    column = df_short[df_short.columns[i]]
    max_value = column.max()
    max_index = column.idxmax()

    if max_value >= thresh and "ROP" in df_short.columns[i] \
    and "<=>" in df_short.columns[i] \
    and not "sensitivity" in df_short.columns[i]:

        max_value_TOF = max_value/surface.site_density
        tof_label = df_short.columns[i].replace("ROP [kmol/m^2 s]","TOF [1/s]")
        imp_rxns.update({tof_label:max_value_TOF})
#         print(df_short.columns[i], "Max = " "{:.10f}".format(max_value_TOF))
        
imp_rxns = {k: v for k, v in sorted(imp_rxns.items(), key=lambda item: item[1], reverse = True)}

for key, value in imp_rxns.items(): 
    print("{:60s} : {:4e} ".format(key,value))

In [ ]:
# get the maximum values
max_dist = 2
imp_rxns = {}
h2 = 0.5
co_co2 = 0.5

df_short = aggdata[(aggdata["T (K)"] == 528) & \
                   np.isclose(aggdata["X_h2 initial"],h2) & \
                   np.isclose(aggdata["CO2/(CO2+CO)"],co_co2)]
df_short
thresh = -1e-15

print(f'Species Steady state ROPs that go below :{thresh}', '\n')

for i in range (15, len(df_short.columns)):

    column = df_short[df_short.columns[i]]
    max_value = column.min()
    max_index = column.idxmin()

    if max_value <= thresh and "ROP" in df_short.columns[i] \
    and "<=>" in df_short.columns[i] \
    and not "sensitivity" in df_short.columns[i]:

        max_value_TOF = max_value/surface.site_density
        tof_label = df_short.columns[i].replace("ROP [kmol/m^2 s]","TOF [1/s]")
        imp_rxns.update({tof_label:max_value_TOF})
#         print(df_short.columns[i], "Max = " "{:.10f}".format(max_value_TOF))
        
imp_rxns = {k: v for k, v in sorted(imp_rxns.items(), key=lambda item: item[1], reverse = False)}

for key, value in imp_rxns.items(): 
    print("{:60s} : {:4e} ".format(key,value))

## Plot potential energy surface

plot the enthalpy vs reaction coordinate

In [ ]:
import cantera as ct
from IPython.display import Image
%matplotlib inline

import os
import sys
import git
import time
sys.path.append(f'/work/westgroup/ChrisB/_01_MeOH_repos/PES_plotter')
sys.path.append(f'/work/westgroup/ChrisB/_01_MeOH_repos/PES_plotter/PyEnergyDiagram')
import pes_plot
import numpy as np

### Hydrogen adsorption/desorption

Hydrogen dissociation vs van der waals adsorption  
the paths available are almost equivalent. H2X is higher energy. to see why it persists we need to examine the later reactions with H2X vs the ones available to H2 

In [ ]:
ppt = pes_plot.pes_plot(cti_file)
species1 = [
    spc_trans["H2"],
    spc_trans["X"],
]

h2_gr = [(spc_trans["H2"],1),(spc_trans["X"],2)]
hx_gr = [(spc_trans["H*"],2)]
h2x_gr = [(spc_trans["H2*"])]

ppt.add_species(species1, 0)
ppt.add_combined_specie(h2_gr, 1)
ppt.add_combined_species([hx_gr, h2x_gr], 3)

ppt.add_reaction("H2(2) + 2 X(1) <=> 2 HX(15)",2)
ppt.add_reaction("H2(2) + X(1) <=> H2X(32)",2)
ppt.plot()


get a list of reactions for CO2 to COOH*(18)

In [ ]:
# get reactants for CO2 to go to carbonyl
reactant = spc_trans["CO*"]
product = spc_trans["COOH*"]
rxn_list = []
reactant_list = []
product_list = []
for rxn in surface.reactions():
    if reactant in rxn.reactants and product in rxn.products:
        rxn_list.append(rxn.equation)
        print(
            rxn.equation, 
            rxn.rate.activation_energy, 
            rxn.rate.pre_exponential_factor,
            rxn.rate.temperature_exponent,
        )
        reactant_list.append([(s,st) for s,st in rxn.reactants.items()])
        product_list.append([(s,st) for s,st in rxn.products.items()])

In [ ]:
ppt = pes_plot.pes_plot(cti_file)
species = [
#     'H*(10)',
    spc_trans["CO*"],
    spc_trans["CO2*"],
]
ppt.add_species(species, 0)
ppt.add_combined_species(reactant_list,1)
ppt.add_reactions(rxn_list, 2)
ppt.add_combined_species(product_list, 3)
ppt.plot()

In [ ]:
reactant = spc_trans["CO*"]
product = spc_trans["HCO*"]
rxn_list = []
reactant_list = []
product_list = []
for rxn in surface.reactions():
    if reactant in rxn.reactants and product in rxn.products:
        rxn_list.append(rxn.equation)
        reactant_list.append([(s,int(st)) for s,st in rxn.reactants.items()])
        product_list.append([(s,int(st)) for s,st in rxn.products.items()])

In [ ]:
if "HCOOH*" in spc_trans.keys():
    reactant = spc_trans["HCOOH*"]
    product = spc_trans["COOH*"]
    rxn_list = []
    reactant_list = []
    product_list = []
    for rxn in surface.reactions():
        if reactant in rxn.reactants and product in rxn.products:
            rxn_list.append(rxn.equation)
            print(
                rxn.equation, 
                rxn.rate.activation_energy, 
                rxn.rate.pre_exponential_factor,
                rxn.rate.temperature_exponent,
            )
            reactant_list.append([(s,int(st)) for s,st in rxn.reactants.items()])
            product_list.append([(s,int(st)) for s,st in rxn.products.items()])
else: 
    print("sorry, no formic acid")

In [ ]:
if "HCOOH*" in spc_trans.keys():
    ppt = pes_plot.pes_plot(cti_file)
    species = [
        spc_trans["COOH*"],
        spc_trans["HCOOH*"],
    ]
    ppt.add_species(species, 0)
    ppt.add_combined_species(reactant_list,1)
    ppt.add_reactions(rxn_list, 2)
    ppt.add_combined_species(product_list, 3)
    ppt.plot()
else:
    print("sorry, no formic acid")

In [ ]:
if "HCOOH*" in spc_trans.keys():
    ppt = pes_plot.pes_plot(cti_file)
    species = [
        spc_trans["HCOOH*"],
        spc_trans["COOH*"],
    ]
    ppt.add_species(species, 0)
    ppt.add_combined_species(reactant_list,1)
    ppt.add_reactions(rxn_list, 2)
    ppt.add_combined_species(product_list, 3)
    ppt.plot()
else:
    print("sorry, no formic acid")

### Make dot file for analyzing reaction network

In [ ]:
# name of output
file_name_dot = "reaction_network_diagrams/big_dot.dot"
file_name_png = file_name_dot.replace(".dot",".png")

# where to find molecule pictures
img_dir = f"./{commit}/species_pictures/"

# beginning and end of digraph
head_str = "graph reaction_paths {\ncenter=1;\n"
tail_str = ' label = "rxn path";\n rankdir=LR\n fontname = "Helvetica";\n}'

# if we want to flag reactions we can color them red
flag_rxn_str = [
    f'{spc_trans["CO*"]} + {spc_trans["OH*"]} <=> {spc_trans["COOH*"]} + {spc_trans["X"]}',
    f'{spc_trans["COOH*"]} + {spc_trans["OH*"]} <=> {spc_trans["CO2*"]} + {spc_trans["H2O*"]}'
]

spec_dot_dict = {}
rxn_dot_list =[]

# combine all species for node making 
all_spec = gas.species_names + surface.species_names

# flag all reactions in which these species participate.  
selec_spec = [
    spc_trans["CO*"],
    spc_trans["COOH*"],
]

# get all species and make them nodes
for node, spec in enumerate(all_spec):
    dot_str = f's{node} [ shape = "box" image="{img_dir+spec+".png"}" label="" width="0.75" height="0.75" imagescale=false fixedsize=false color="none" ];\n'
    spec_dot_dict.update({spec:[f's{node}', dot_str]})
    
# combine all reactions so we can iterate over all of them
all_reactions = gas.reactions() + surface.reactions()
       
# make connections for each reactant species.
for rxn in all_reactions:
    for reac in selec_spec:
        if reac in rxn.reactants:
            for prod in rxn.products.keys():
                reac_s = spec_dot_dict[reac][0]
                prod_s = spec_dot_dict[prod][0]
                mod_rxn_eq = rxn.equation.replace(" ", "") #.replace("<=>","\n<>\n")
                
                if rxn.equation in flag_rxn_str:
                    dot_str = f'{reac_s} -- {prod_s} [fontname="Helvetica", color="red", width = 3.0, penwidth=1.0, arrowsize=1.0, label="{mod_rxn_eq}"];\n'
                else: 
                    dot_str = f'{reac_s} -- {prod_s} [fontname="Helvetica", width = 3.0, penwidth=1.0, arrowsize=1.0, label="{mod_rxn_eq}"];\n'
                
                # don't add duplicates
                if dot_str not in rxn_dot_list:
                    rxn_dot_list.append(dot_str)

# remove species that do not participate in the set of reactions                    
for spec in list(spec_dot_dict.keys()):
    spec_search_str = f'{spec_dot_dict[spec][0]+" "}'
    if not any(spec_search_str in i for i in rxn_dot_list):
        del spec_dot_dict[spec]
                  
with open(file_name_dot, "w") as f:
    # add head
    f.write(head_str)
    
    # add reactions
    for connect in rxn_dot_list:
        f.write(connect)
    for value in spec_dot_dict.values():
        f.write(value[1])
    
    # add tail
    f.write(tail_str)

os.system(f"dot {file_name_dot} -Tpng -o {file_name_png} -Gdpi=300")

Image(file_name_png,width = 1500, unconfined=True)